In [ ]:
dir_root = '/user/centos/'
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import Window, Row
from pyspark.sql import functions as sf

In [ ]:
ddf_temp = spark.createDataFrame([Row(mid=1, month=1.0, temperature=3.0),
 Row(mid=2, month=1.0, temperature=6.0),
 Row(mid=3, month=2.0, temperature=4.0),
 Row(mid=4, month=3.0, temperature=8.0),
 Row(mid=5, month=3.0, temperature=9.0),
 Row(mid=6, month=3.0, temperature=8.0),
 Row(mid=7, month=3.0, temperature=12.0)])

### *Exercise A*
2. Add a column with the average temperature of the month
3. Compute the temperature delta with the previous measurement
1. Exclude rows of months with an average temperature below 5 degrees 

In [ ]:
# 1.
wspec = Window.partitionBy('month')
ddf_temp = ddf_temp.withColumn('mean_temp_month', sf.mean('temperature').over(wspec))
ddf_temp.show()

In [ ]:
# 2.
wspec = Window.orderBy('mid')
ddf_temp = ddf_temp.withColumn('temp_delta', sf.col('temperature') - sf.lag('temperature').over(wspec))
ddf_temp.show()

In [ ]:
# 3.
ddf_temp.filter(sf.col('mean_temp_month') > 5).show()

### *Exercise B*
1. Demean the flight delays partitioning by year;
2. Demean the flight delays partitioning by year/carrier;
3. For each year, find the carriers with the most flights cancelled;
4. Same as 3., but normalize by number of flights;
5. Per airline, find the airport with the most delays due to security reasons in a given year/month.

In [ ]:
ddf_air = spark.read.load(dir_root + 'data/airlines.parquet')

In [ ]:
# 1.
window = (Window.partitionBy(ddf_air['year']))

(ddf_air.dropna(subset='arr_delay')
        .select('year', 'carrier', ddf_air['arr_delay'] - sf.avg('arr_delay').over(window))).show(5)

In [ ]:
# 2.
window = (Window.partitionBy(ddf_air['year'], ddf_air['carrier']))
(ddf_air.dropna(subset='arr_delay')
        .select('year', 'carrier', ddf_air['arr_delay'] - sf.avg('arr_delay').over(window))).show(5)

In [ ]:
# 3
cancelled_ddf_air = (ddf_air.dropna(subset='arr_cancelled')
                              .select('year', 'carrier', 'arr_cancelled')
                              .groupby('year', 'carrier').agg({'arr_cancelled': 'sum'})
                              .withColumnRenamed('sum(arr_cancelled)', 'cancelled'))

window = (Window.partitionBy(cancelled_ddf_air['year'])).orderBy(cancelled_ddf_air['cancelled'].desc())
ranked_c_ddf_air = cancelled_ddf_air.select('year', 'carrier', 'cancelled', sf.rank().over(window).alias('n'))

ranked_c_ddf_air.filter(ranked_c_ddf_air['n'] == 1).show(5)

In [ ]:
# 4
cancelled_ddf_air = (ddf_air.dropna(subset='arr_cancelled')
                              .select('year', 'carrier', 'arr_cancelled', 'arr_flights')
                              .groupby('year', 'carrier').agg({'arr_cancelled': 'sum', 'arr_flights': 'sum'})
                              .withColumnRenamed('sum(arr_cancelled)', 'cancelled')
                              .withColumnRenamed('sum(arr_flights)', 'flights')
                              .selectExpr('year', 'carrier', 'cancelled/flights')
                              .withColumnRenamed('(cancelled / flights)', 'cancelled_pct'))

window = (Window.partitionBy(cancelled_ddf_air['year'])).orderBy(cancelled_ddf_air['cancelled_pct'].desc())
ranked_c_ddf_air = cancelled_ddf_air.select('year', 'carrier', 'cancelled_pct', sf.rank().over(window).alias('n'))

ranked_c_ddf_air.filter(ranked_c_ddf_air['n'] == 1).show(5)

In [ ]:
# 5. Per airline, find the airport with the largest delay. Report the year and month in which this happens
window = (Window.partitionBy(sf.col('carrier')).orderBy(sf.col('security_delay').desc()))
sec = ddf_air.dropna(subset='security_delay').select('carrier', 'airport', 'security_delay', 'year', 'month', sf.rank().over(window).alias('n'))
sec.filter(sec['n'] == 1).show()